# classification.core

> Fill in a module description here

In [ ]:
#| default_exp classification.core

In [ ]:
#| export 
from monitosed.data.core import *
import numpy as np
import pandas as pd
from fastcore.basics import store_attr
from einops import rearrange, repeat
import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
#| export
def get_users_labels(path, label_col='R_Pain_Int', drop_ixs=None):
    data = pd.read_excel(path)
    labels = data['R_Pain_Int'].dropna()
    users = data['ID'].dropna()
    users.drop(drop_ixs, inplace=True)
    labels.drop(drop_ixs, inplace=True)
    return users, labels

In [ ]:
users, labels = get_users_labels('../_data/Smarthy2_Behavioral.xlsx', drop_ixs=[0,2,7, 12, 13, 16])

In [ ]:
#| export
def stack_trials(mats, signal_len=None):
    data = prepare_train_data(mats)
    if signal_len: data = data[..., :signal_len]
    x = torch.from_numpy(np.concatenate(data, axis=0)).unsqueeze(1)
    return torch.tensor(x, dtype=torch.float32)

In [ ]:
#| export
def prepare_labels(mats, labels):
    read_mats = [read_data(mat) for mat in mats]
    rearranged_mats = [new_rearrange(mat) for mat in read_mats]
    y = np.concatenate([np.repeat(labels[i], rearranged_mats[i].shape[0]) for i in range(len(mats))])
    return torch.tensor(y, dtype=torch.float32)

In [ ]:
#| export
class CustomLoss(nn.Module):
    def __init__(self, size_average=None, reduce=None, alpha=0.5, reduction: str = 'mean') -> None:
        super(CustomLoss, self).__init__()
        store_attr()

    def forward(self, input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        l1 = F.l1_loss(input, target, reduction=self.reduction)
        mse = F.mse_loss(input, target, reduction=self.reduction)
        return self.alpha*l1 + (1-self.alpha)*mse

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()